# ENTREGÁVEIS OFICIAIS:

Atendendo aos requisitos de coleta, ingestão, armazenamento, transformação, consumo e orquestração.

**2) Uso de ferramentas de dados não-relacionais estruturados (SQL) para consultas.**

Aqui o arquivo pós transformação e tratamento podem ser consumidos direto do data lake (storage) ou pode ser carregado através da ferramenta de ETL para o Ambiente Não Relacional.

- Importe o arquivo final da etapa anterior para um ambiente de análise como: Hive, Redshift, BigQuery, Synapse, Athena ou semelhante.

- Usando o (Hive, Synapse, Redshift, Athena, Big Query e etc) faça uma análise exploratória dos seus dados:
a) contagem de registros
b) visualize uma amostragem dos dados (as dez primeiras linhas).
c) descrição dos campos da tabela
d) contagem de valores distintos por campo chave
e) estatística descritiva (média, mínimo, máximo)
f) distribuição de frequência (contagem dos elementos de cada campo)

- Prepare um relatório de qualidade de dados. Inclua a taxa de campos nulos, tipos inconsistentes e campos redundantes.

- Elabore um pequeno dashboad (Power BI, Tableau, Apache Superset ou Matplotlib/Seaborn) com gráfico de tendencias, distribuição de categorias e KPIs relevantes.

In [0]:
spark.sql("CREATE CATALOG IF NOT EXISTS bus")
spark.sql("CREATE SCHEMA IF NOT EXISTS bus.bronze")
spark.sql("CREATE SCHEMA IF NOT EXISTS bus.silver")
spark.sql("CREATE SCHEMA IF NOT EXISTS bus.gold")

In [0]:
spark.sql("USE CATALOG bus")
spark.sql("USE SCHEMA bronze")

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS bus.bronze.rw

In [0]:
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("/Volumes/bus/bronze/rw/dados_mockados.csv")

df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("bus.bronze.ota_bus_ticket_sales")

In [0]:
%sql
SELECT * FROM bus.bronze.ota_bus_ticket_sales LIMIT 10

# Análise Exploratória dos Dados

In [0]:
%sql
SELECT COUNT(*) FROM bus.bronze.ota_bus_ticket_sales

In [0]:
%sql
SELECT * FROM bus.bronze.ota_bus_ticket_sales LIMIT 10

In [0]:
%sql
DESCRIBE TABLE bus.bronze.ota_bus_ticket_sales

In [0]:
df_ota_bus_ticket_sales = spark.table("bus.bronze.ota_bus_ticket_sales")

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import StringType, DoubleType, IntegerType, LongType, FloatType
import pandas as pd

In [0]:
contagem_distinta = df_ota_bus_ticket_sales.select([
    countDistinct("nk_ota_localizer_id").alias("distinct_localizer_id"),
    countDistinct("fk_contact").alias("distinct_contacts"),
    countDistinct("place_origin_departure").alias("distinct_origin_departure")
])

display(contagem_distinta)

In [0]:
df_ota_bus = df_ota_bus_ticket_sales.withColumn("gmv_success", regexp_replace("gmv_success", ",", ".").cast("float"))

df_ota_bus_final = df_ota_bus.select(
    mean("gmv_success").alias("media_gmv"),
    min("gmv_success").alias("min_gmv"),
    max("gmv_success").alias("max_gmv"),
    mean("total_tickets_quantity_success").alias("media_tickets"),
    min("total_tickets_quantity_success").alias("min_tickets"),
    max("total_tickets_quantity_success").alias("max_tickets")
)

display(df_ota_bus_final)

In [0]:
df_ota_bus_ticket_sales.groupBy("fk_departure_ota_bus_company").count().orderBy("count", ascending=False).show()

In [0]:
df_ota_bus_ticket_sales.groupBy("place_origin_departure").count().orderBy("count", ascending=False).show()

In [0]:
%sql
SELECT
  *
FROM
  bus.bronze.ota_bus_ticket_sales LEFT JOIN bus.bronze.dim_clientes USING (fk_contact)
WHERE
  active_record_indicator = 'S'
LIMIT 10

# Relatório de Qualidade

In [0]:
df_quality_report = spark.table("bus.bronze.ota_bus_ticket_sales")

schema = df_quality_report.dtypes
print("Estrutura da Tabela:")
for column, dtype in schema:
    print(f"Coluna: {column}, Tipo de Dado: {dtype}")

In [0]:
null_counts = df_quality_report.select([F.sum(F.col(c).isNull().cast("int")).alias(c) for c in df.columns])
null_counts.display()

In [0]:
missing_nk_ota_localizer_id = df_quality_report.filter(F.col("nk_ota_localizer_id").isNull())
missing_nk_ota_localizer_id.display()

In [0]:
duplicados = df_quality_report.count() - df_quality_report.distinct().count()
print(f"Quantidade de duplicatas: {duplicados}")

In [0]:
# Verificando se houve o retorno de vendas com valores negativos

df_precos_invalidos = df_quality_report.filter(F.col("gmv_success") < 0)
df_precos_invalidos.display()

In [0]:
unique_counts = {col: df_quality_report.select(col).distinct().count() for col in df_quality_report.columns}
print("Contagem de valores únicos por coluna:")
for col, count in unique_counts.items():
    print(f"Coluna: {col}, Valores únicos: {count}")

In [0]:
describe_quality_report = df_quality_report.describe().toPandas()
print("Estatísticas Descritivas:")
print(describe_quality_report)

quality_report = {
    "Estrutura": schema,
    "Valores Nulos": null_counts.toPandas(),
    "Duplicatas": duplicates,
    "Valores Inconsistentes (Preço Negativo)": df_invalid_prices.count(),
    "Valores Únicos por Coluna": unique_counts,
    "Estatísticas Descritivas": descriptive_stats,
    "Dados Ausentes em 'departure_date'": missing_departure_dates.count()
}

print("Relatório gerado com sucesso!")